# Elementy programowania funkcyjnego w Pythonie
Programowanie funkcyjne to takie, w którym zamiast dawać przepisy jak kolejno zmieniać stan obiektów, aby dojść do stanu pożądanego (to jest esencja programowania imperatywnego), dajemy przepisy jak konstruować nowe wartości ze starych (zwykle używając funkcji), aby dojść do wartości pożądanej. Elementami tego paradygmatu w Pythonie są niezmienialne struktury danych, takie jak napisy, krotki, frozen_set-y itd, ale także dostępność niektórych idiomów programistycznych, charakterystycznych dla "prawdziwych" języków funkcyjnych, takich jak Haskell czy OCaml.

W tym scenariuszu (i następnym) poznamy niektóre z tych idiomów, część z nich jest podobna do tego, co jest dostępne w Javie.

## Funkcja map

Mając daną listę czasem potrzebujemy przetworzyć jej elementy i wyniki tego przetworzenia zebrać do nowej listy. Załóżmy, że potrzebujemy każdemu napisowi na liście przypisać największą liczbę parzystą niewiększą niż długość tego napisu.

In [4]:
def prawie_długość(s):
    l = len(s)
    if l % 2 == 1:
        return l - 1
    else:
        return l
print(prawie_długość("ala"), prawie_długość("kota"))

2 4


Samo przerabianie listy napisów na odp. listę liczb można w Pythonie zrobić na (co najmniej :) trzy różne sposoby.
### Sposób 1 (tradycyjnie)

In [5]:
lista_napisów = "Ala zaadaptowała strasznie zabawnego kota".split()
lista_napisów

['Ala', 'zaadaptowała', 'strasznie', 'zabawnego', 'kota']

In [6]:
l = []
for s in lista_napisów:
    l.append(prawie_długość(s))
l

[2, 12, 8, 8, 4]

Ten sposób ma trzy linijki, z czego mniej więcej dwie to boilerplate code, czyli kod "nudny, niewiele wnoszący, powtarzalny, zaśmiecający i zaciemniający czytelność [...]. Mimo to nie można z niego zrezygnować, ponieważ jest zwyczajnie potrzebny, łącząc w spójną całość pozostałe fragmenty aplikacji." (żródło: [pierwszy link z google'a](https://stormit.pl/boilerplate-code/)).

Sposób ten jest bardzo imperatywny w swej naturze - każdy obrót pętli zmiania stan obiektu (listy) przypisanego do zmiennej `l`.

### Sposób 2 (list comprehension)

In [7]:
[prawie_długość(s) for s in lista_napisów]

[2, 12, 8, 8, 4]

Tutaj już boilerplate ma tylko ⅓ linijki, więc jest całkowicie akceptowalny. Już na pierwszy rzut oka widać, co ten kod robi. Nie ma tu też żadnej "zmiany stanu", po prostu magicznie powstaje lista wynikowa o odpowiednio skonstruowanych elementach.
### Sposób 3 (funkcja map)

In [8]:
list(map(prawie_długość, lista_napisów))

[2, 12, 8, 8, 4]

Pierwszym argumentem `map` jest funkcja, która będzie zaaplikowana do każdego elementu drugiego argumenty (czyli listy napisów). Ale tak naprawdę drugi argument wcale nie musi być listą, byleby miał elementy, które można z niego "wyciągać" (tak jak np. `range(...)`, krotka, itp.). Wynikiem wywołania funkcji `map` też nie jest lista, tylko iterator, podobny do np. `range(...)` (co ma swoje zalety i wady), z którego można zrobić listę, używając konwersji `list(...)`. Funkcja `map` jest jednym z najbardziej emblematycznych idiomów programowania funkcyjnego.

W naszym przykładzie, funkcję, którą chcemy zaaplikować do każdego elementu listy, zdefiniowaliśmy wcześniej (zajęło nam to całe 6 linijek ;) ale często operacja, którą chcemy zastosować jest albo bardzo prosta, albo zdefiniowana, ale "nie do końca". Na przykład jeśli chcielibyśmy policzyć liczbę wystąpień literki `a` w poszczególnych napisach z listy, to mamy gotową funkcję `count`, ale musimy do niej dołożyć informację, że chodzi o literkę `a`. Tradycyjny sposób to oczywiście zdefiniowanie odpowiedniej funkcji:

In [9]:
def licz_a(s):
    return s.count('a')

Ale można też użyć funkcji anonimowej, czyli tzw. *wyrażenia lambda*.
## Funkcje anonimowe

In [10]:
list(map(lambda s: s.count('a'), lista_napisów))
# funkcja - obiekt

[1, 5, 1, 2, 1]

Funkcje anonimowe zajmują chyba pierwsze miejsce jeśli chodzi o skojarzenia z programowaniem funkcyjnym.  
Składnia wyrażeń lambda w Pythonie to `lambda <parametry>: <wyrażenie>`, gdzie `<parametry>` to lista nazw parametrów rozdzielonych przecinkami, które to nazwy mogą oczywiście występować w `<wyrażeniu>`. Zdefiniowana funkcja jest taka jak w definicji:
```
def anonimowa(<parametry>):
    return <wyrażenie>
```

### Ćwiczenie 1
Oblicz jeszcze raz listę "prawie długości" napisów na `liście_napisów` przy użyciu funkcji `map`, zapisując własną wersję funkcji `prawie_długość` za pomocą wyrażenia lambda.

In [16]:
list(map(lambda s: len(s)-1 if len(s) % 2 == 1 else len(s), lista_napisów))

[2, 12, 8, 8, 4]

----------
Z punktu widzenia Pythona wyrażenia lambda są po prostu wyrażeniami, czyli mogą występować we wszystkich fragmentach programu, tam gdzie pozostałe wyrażenia np. `x+5`. Funkcje reprezentowane przez wyrażenia lambda (a także na wszystkie inne sposoby, np. poprzez nazwy funkcji zdefiniowanych za pomocą `def`) są traktowane przez Python jak wszystkie inne obiekty: można je przypisywać na zmienne, przekazywać do innej funkcji, dawać jako wynik funkcji, trzymać na liście albo w innych strukturach danych, itd.

In [23]:
import random

def przez2(x):
    return x//2

operacje = [przez2, lambda x: x+1, lambda x: x-1, lambda x: 2*x]

def losuj_operację():
    los = random.randrange(5)
    print("Wylosowano:", los)
    if los < len(operacje):
        return operacje[los]
    else:
        return lambda x: 0

In [24]:
# za każdym odpaleniem losujemy funkcję, ale tę samą dla wszystkich liczb z range
list(map(losuj_operację(), range(10)))

Wylosowano: 2


[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

In [10]:
# dla każdej liczby z range osobne losowanie operacji
list(map(lambda x: operacje[random.randrange(4)](x), range(10)))

[-1, 2, 3, 6, 8, 2, 12, 6, 7, 8]

*Dygresja*: `random.randint(x,y)` to "jedyna" funkcja w Pythonie, która traktuje przedział liczb całkowitych jako obustronnie domknięty. Dlatego tu użyliśmy funkcji `random.randrange`, która przyjmuje parametr(y) tak jak `range`.

### Różne użyteczne rzeczy w lambdach i nie tylko
#### Wyrażenie warunkowe
Bardziej użyteczną rzeczą od losowania funkcji :) jest *wyrażenie warunkowe* (to nie jest to samo, co *instrukcja* warunkowa, którą poznaliśmy wcześniej). Wyrażenie warunkowe w Pythonie ma składnię:

`<wyrażenie1> if <warunek> else <wyrażenie2>`  

co na polski tłumaczylibyśmy `<wyrażenie1>` o ile `<warunek>` wpp `<wyrażenie2>`. 

W C/C++/Javie byłoby to oczywiście `(<warunek> ? <wyrażenie1> : <wyrażenie2>)`. W Pythonie kolejność elementów jest na pierwszy rzut oka trochę dziwna, ale ta konstrukcja przewidziana jest na sytuacje, gdy warunek raczej jest spełniony :) np:

In [25]:
list(map(lambda x: 'a'*x if x else "zero!", range(5)))

['zero!', 'a', 'aa', 'aaa', 'aaaa']

To "dziwna" kolejność elementów wyrażenia warunkowego ma dwie zalety: po pierwsze jest symetryczna (estetyka ;) a po drugie dobrze się komponuje sama ze sobą w przypadku więcej niż jednego warunku:

In [26]:
x = 42
print("ujemna" if x < 0 else "zero" if x == 0 else "dodatnia")

dodatnia


Oczywiście wyrażenie warunkowe wcale nie musi występować w lambdzie. Ale tu jest szczególnie użyteczne, bo w "ciele" lambdy nie może być instrukcji.

Oczywiście jak się bardzo chce, to każde ograniczenie można obejść, choćby poprzez wywołanie funkcji. 

#### Jak zrobić print w lambdzie?
Prostym (i często przydatnym np. do debugowania) sposobem obchodzenia ograniczenia składniowego wyrażeń lambda jest tworzenie "tymczasowych" krotek, np:

In [27]:
list(map(lambda x: (print("Tu lambda:", x), x+1)[1], range(5)))

Tu lambda: 0
Tu lambda: 1
Tu lambda: 2
Tu lambda: 3
Tu lambda: 4


[1, 2, 3, 4, 5]

W ciele lambdy tworzymy na chwilę krotkę, której elementem o indeksie 0 jest *wynik* funkcji `print` (czyli `None`), a elementem o indeksie 1 jest właściwa wartość, którą wybieramy z krotki jako wynik całej lambdy. Inną konstrukcją o podobnej funkcji jest np. operator alternatywy: `print(x) or x+1` również spełni nasze oczekiwania, ponieważ `None` zwrócony przez `print` jest traktowane jak fałsz, a w tym przypadku wartość alternatywy to wprost wartość drugiego członu (wcale niekoniecznie `True` lub `False`).

Należy jednak pamiętać, że elementy programowania funkcyjnego, w tym w szczególności lambdy, nie są stworzone dla efektów ubocznych, a dla obliczania (nowych) wartości na podstawie argumentów. A zatem używanie efektów ubocznych w tych elementach to raczej zły styl programowania.

### Ćwiczenie 2
Wyznacz z pomocą `map` reszty z dzielenia przez 17 kwadratów kolejnych liczb naturalnych mniejszych niż 20. 

In [32]:
list(map(lambda n: (n**2)%17, range(20)))

[0, 1, 4, 9, 16, 8, 2, 15, 13, 13, 15, 2, 8, 16, 9, 4, 1, 0, 1, 4]

## Funkcja filter

Drugą najpopularniejszą funkcją wyższego rzędu (czyli funkcją, której parametrem jest funkcja) na listach jest `filter`. Służy ona do wybierania z listy elementów spełniających podany predykat, np.

In [33]:
def czy_pal(n):
    '''Sprawdza czy reprezentacja tekstowa liczby n jest palindromem.'''
    s = str(n)
    return s == s[::-1]  # nie do końca efektywne, ale krótkie :)

lista_liczb = [1, 10, 11, 121, 1000]

In [34]:
list(filter(czy_pal, lista_liczb))

[1, 11, 121]

Pierwszym argumentem `filter` jest funkcja (nazwana poprzednio predykatem), która aplikowana jest do kolejnych elementów drugiego argumentu - listy (czy czegoś podobnego) - i wynikiem są tylko te elementy, dla których użyta funkcja daje wynik, który Python traktuje jako "prawda" (zwykle po prostu `True`). 

Podobnie jak w przypadku `map`, wynikiem funkcji `filter` jest iterator (dlatego w przykładach używaliśmy `list(filter(...))`. I znów, podobnie jak w przypadku `map`, użycie funkcji `filter` również można zastąpić innymi technikami: 

In [ ]:
# tradycyjnie... prawie 3 linie boilerplate code
l = []
for n in lista_liczb:
    if czy_pal(n):
        l.append(n)
l

[1, 11, 121]

In [ ]:
# list comprehension, prawie tak samo zgrabnie jak filter
[n for n in lista_liczb if czy_pal(n)]

[1, 11, 121]

### Ćwiczenie 3
Wyznacz wszystkie liczby, które są kwadratami liczb naturalnych mniejszych niż 100 i mają w swojej reprezentacji dwie jedynki pod rząd.

In [56]:
list(filter(lambda s: str(s).find('1', str(s).find('1')+1) == str(s).find('1')+1 , map(lambda n: n**2, range(100))))

[1156, 2116]

## Po co map i filter?
Oczywistym jest, że bez `map` i `filter` można żyć. W zasadzie list comprehension to takie połączenie `map` i `filter` i w dodatku dużo osób twierdzi, że list comprehension jest bardziej czytelne, a w większości przypadków również bardziej efektywne. Zaletą `map` i `filter` jest zwięzłość (zwłaszcza w przypadku gdy funkcje stosowane jako parametr już istnieją) oraz łatwość komponowania.

Poeksperymentujmy z następującym problemem: podnieśmy do kwadratu wszystkie liczby z jakiegoś dużego przedziału (np. od 0 do 5 milionów), jeśli fortuna nam będzie sprzyjała podnieśmy je jeszcze raz do kwadratu, a następnie wybierzmy z tego te liczby, których zapis dziesiętny jest palindromem.

Kod ten, zapisany za pomocą `map` i `filter` będzie wyglądał następująco:

Zauważmy, że ten kod do przedostatniej linijki włącznie, wykonuje się natychmiastowo. Cała praca związana z przetwarzaniem długiej listy odbywa się w ostatniej linii. Ten sam kod z użyciem list comprehension tworzyłby jedną lub dwie pośrednie listy pomocnicze (można to zobaczyć podglądając intepreter Pythona za pomocą polecenia top). Można uniknąć tworzenia list pomocniczych, używając wyrażeń-generatorów, które produkują obiekty analogiczne do tych produkowanych przez `map` i `filter`: 

In [ ]:
l = range(5_000_000)
l = (x*x for x in l)
if random.choice([True, False]):
  l = (x*x for x in l)
[x for x in l if czy_pal(x)]

[0,
 1,
 14641,
 104060401,
 1004006004001,
 10004000600040001,
 100004000060000400001,
 1000004000006000004000001]

W tym kodzie również cała praca zostanie wykonana w ostatniej linijce, trudno też mu cokolwiek zarzucić, jeśli chodzi o przejrzystość i łatwość komponowania. Pomijając losowanie, "tekstowe" złożenie kilku takich operacji przetwarzających listę nieco lepiej wygląda przy użyciu list comprehension:

In [ ]:
list(filter(czy_pal, map(lambda x: x**4, range(5_000_000))))

[0,
 1,
 14641,
 104060401,
 1004006004001,
 10004000600040001,
 100004000060000400001,
 1000004000006000004000001]

In [ ]:
[x**4 for x in range(5_000_000) if czy_pal(x**4)]

[0,
 1,
 14641,
 104060401,
 1004006004001,
 10004000600040001,
 100004000060000400001,
 1000004000006000004000001]

Trzeba przy tym zaznaczyć, że byliśmy zmuszeni do dwukrotnego zapisania (i czasem też obliczenia) wyrażenia `x**4` (choć prawdę mówiąc dałoby się tego uniknąć, ale z użyciem konstrukcji, które są całkowicie niezgodne z duchem programowania funkcyjnego, którego fragmenty właśnie poznajemy - dlatego to teraz pomijamy :).

Gdyby zależało nam na poznaniu nie tylko palindromu, ale też liczby, której czwarta potęga jest palindromem, modyfikacja w wersji list comprehension jest łatwiejsza, choć i w przypadku `map` i `filter` radzimy sobie bez problemu używając par.

In [ ]:
list(filter(lambda p: czy_pal(p[1]), map(lambda x: (x, x**4), range(5_000_000))))

[(0, 0),
 (1, 1),
 (11, 14641),
 (101, 104060401),
 (1001, 1004006004001),
 (10001, 10004000600040001),
 (100001, 100004000060000400001),
 (1000001, 1000004000006000004000001)]

In [ ]:
[(x, x**4) for x in range(5_000_000) if czy_pal(x**4)]

[(0, 0),
 (1, 1),
 (11, 14641),
 (101, 104060401),
 (1001, 1004006004001),
 (10001, 10004000600040001),
 (100001, 100004000060000400001),
 (1000001, 1000004000006000004000001)]

Reasumując, nie widać specjalnie przewagi `map` i `filter` nad list comprehension, zwłaszcza jeśli pamiętamy, że w przypadku wielu kroków wcale nie musimy generować list pośrednich (możemy używać wyrażeń-generatorów). W dodatku funkcje te uznawane są przez niektórych za *niepytoniczne* (ang. unpythonic :)
Zainteresowanych odsyłam do dość obszernego wątku na [stackoverflow](https://stackoverflow.com/questions/1247486/list-comprehension-vs-map).

Jeśli chodzi o efektywność, to to jak zwykle zależy. W skrócie, jeśli mamy do wykonania jedną operację i w wyniku potrzebujemy listy, to lepiej używać list comprehension, z wyjątkiem `map(gotowa_funkcja, lista)`. Z kolei używanie wyrażeń generatorów (a konkretnie generowanie z nich końcowych list) jest wyraźnie wolniejsze niż `map` i `filter`. W skrócie: warto te funkcje znać, ale nie ma po co ich zbyt często używać.

## Ciekawostki o map, filter i nie tylko

* Funkcje `map` i `filter` nie produkują list, tylko iteratory. Trzeba z nimi ostrożnie, bo są trochę podobne do list, co może dawać fałszywe poczucie komfortu, a niektóre różnice mogą być zaskakujące:

In [ ]:
l = [x*x for x in range(7)]
print("Pierwszy raz (:", *l, ":)")
print("Drugi raz    (:", *l, ":)")

Pierwszy raz (: 0 1 4 9 16 25 36 :)
Drugi raz    (: 0 1 4 9 16 25 36 :)


In [ ]:
l = map(lambda x: x*x, range(7))
print("Pierwszy raz (:", *l, ":)")
print("Drugi raz    (:", *l, ":)")

Pierwszy raz (: 0 1 4 9 16 25 36 :)
Drugi raz    (: :)


Okazuje się, że iterator stworzony przez `map` jest "jednorazowy" (jak Streamy w Javie). Można z niego przeczytać wartości w celu utworzenia listy, albo iterowania pętlą, ale po przeczytaniu wszystkich wartości "znikają" one z iteratora. Zauważmy, że to działa podobnie do wczytywania danych z plików wejściowych (w szczególności ze strumieni takich jak np. standardowe wejście).

Zaletą iteratorów jest to, że można tanio "utworzyć" bardzo długi ciąg, a potem skorzystać tylko z takiej części, jaka jest potrzebna. Niestety, dla iteratorów nie działa przyjemna składnia do wybierania fragmentów (slice'ów) - trzeba użyć odpowiedniej funkcji z biblioteki `itertools`.

In [ ]:
bardzo_długi_ciąg = map(lambda x: x*x, range(10**100)) # dziesięć do setnej :)

import itertools
list(itertools.islice(bardzo_długi_ciąg,0,10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

* Jeśli chcesz odsiać "nieprawdziwe" elementy z listy za pomocą `filter`, zamiast funkcji można podać `None`.

In [ ]:
dziwna_lista = "ala   ma    kota".split(" ")
print(dziwna_lista)

['ala', '', '', 'ma', '', '', '', 'kota']


In [ ]:
list(filter(lambda x: x, dziwna_lista))

['ala', 'ma', 'kota']

In [ ]:
list(filter(None, dziwna_lista))

['ala', 'ma', 'kota']

* Funkcję `map` można używać również z kilkoma listami i wieloargumentową funkcją:

In [ ]:
list(map(lambda x,y: x+y, [1,2,3], [40,30,20,10]))

[41, 32, 23]

Ten sam efekt można uzyskać za pomocą następującego wyrażenia list comprehension:

In [ ]:
[x + y for (x,y) in zip([1,2,3], [40,30,20,10])]

[41, 32, 23]

Użyta powyżej operacja `zip` tworzy iterator par złożonych z elementów obu "list", dopóki jedna z list się nie skończy.

In [ ]:
list(zip(range(10), range(100,50,-10)))

[(0, 100), (1, 90), (2, 80), (3, 70), (4, 60)]

* Wyrażenie lambda może mieć zero argumentów. Do czego to może służyć? Nie do `map` albo `filter`, ale na przykład do podania akcji do wykonania w specyficznych okolicznościach.

In [ ]:
def rób_coś(akcja, w_razie_awarii):
    print("Akcja! ...", end="")
    if akcja:
        print("OK.")
    else:
        w_razie_awarii()

rób_coś(akcja = True, w_razie_awarii = lambda : print("AWARIA!!!"))

rób_coś(akcja = False, w_razie_awarii = lambda : print("AWARIA!!!"))

Akcja! ...OK.
Akcja! ...AWARIA!!!


## Funkcja reduce

Kolejną ważną funkcją wyższego rzędu na listach jest `reduce`. Nie ma jej wśród funkcji dostępnych "od razu", jest w module `functools`.

In [60]:
from functools import reduce
reduce(lambda x,y: x*y, [2,10,1,7])

140

Działanie tej funkcji najlepiej wyobrazić sobie w ten sposób, że "do listy" zamiast przecinków wstawiamy * (tym razem to po prostu mnożenie :) i usuwamy `[` klamerki `]`, a zatem otrzymujemy 2 * 10 * 2 * 7. Oczywiście takie tłumaczenie jest dobre tylko dla funkcji łącznej (czyli takiej, której jest wszystko jedno jak wstawimy nawiasy). W przypadku innej funkcji możemy się przekonać, że nawiasy wstawione są "bardziej z lewej".

In [61]:
reduce(lambda x,y: x-y, [567,100,30,1])

436

Wynik to 567-100-30-1, czyli ((567-100)-30)-1. Nazwa `reduce` wzięła się stąd, że dokonujemy "redukcji" dwóch wartości do jednej - np. poprzez ich pomnożenie lub odjęcie - i postępujemy tak aż do skutku, czyli aż z całego zestawu zostanie jedna wartość.

Ale funkcji `reduce` można dać jeszcze jeden parametr - wartość startową.

In [62]:
reduce(lambda x,y: x-y, [567,100,30,1], 999)

301

Teraz wynik to (((999-567)-100)-30)-1. Przy okazji, jak podamy wartość startową, funkcja ma sensowną wartość (właśnie tą startową) dla pustej listy.

Nic nie stoi też na przeszkodzie, aby wartość początkowa była zupełnie innego typu niż elemety listy, oczywiście nasza funkcja musi to brać pod uwagę.

In [63]:
reduce(lambda s,x: s | {x},  [20,8,8,1,20],  {1000,1001}) 

{1, 8, 20, 1000, 1001}

W powyższym przykładzie `|` oznacza operator sumy zbiorów, a wynik to efekt dodawania wszyskich elementów listy do początkowego zbioru. 

Oczywiście każde działanie robione za pomocą `reduce` można zaprogramować za pomocą pętli po liście i zmieniających się wartości zmiennych, ale - zwłaszcza jak funkcja jest krótka - rozwiązaniom z `reduce` nie można odmówić zwięzłości i elegancji :)

### Ćwiczenie 4
Napisz funkcję `silnia` za pomocą `reduce`.

In [65]:
from functools import reduce

def silnia(n):
    return reduce(lambda x,y: x*y, range(1,n+1))

### Ćwiczenie 5
Napisz za pomocą `reduce` funkcję `sumuj`, która dla danej listy liczb oblicza parę, której pierwsza współrzędna będzie sumą liczb parzystych z tej listy, a druga sumą liczb nieparzystych. Może przydać się tutaj wyrażenie warunkowe (patrz wyżej). Przykłady: `sumuj [1,2,3,4,5] = (6,9)`, `sumuj [1,3,5] = (0,9)`

In [77]:
def sumuj(lista):
    return (reduce(lambda x,y: x+y, filter(lambda n: n%2==0, lista)), reduce(lambda x,y: x+y, filter(lambda n: n%2==1, lista)))
    

## Funkcje opcjonalnie wyższego rzędu

Niektóre funkcje działające na ciągach, np. `max`, `min`, `sorted`, mają opcjonalny parametr `key`, który jest funkcją "wyciągającą" rzecz do porównania z danego elementu. Tam też wyrażenie lambda może być pomocne. 

In [67]:
print(min(lista_napisów, key=len))
print(sorted("Grzegorz Brzęczyszczykiewicz Chrząszczyżewoszyce powiat Łękołody".split(), 
             key=lambda s: sum(x.lower() in 'aąeęioóuy' for x in s)))
print(max([(1, 2.5), (5, 0.7), (8, 1.5)], key=lambda p: p[1]))


Ala
['Grzegorz', 'powiat', 'Łękołody', 'Brzęczyszczykiewicz', 'Chrząszczyżewoszyce']
(1, 2.5)


# Zadanie dnia
Napisz program wczytujący plik `.ipynb` (tak naprawdę w formacie json), którego nazwa zostanie podana jako argument wywołania programu. Po wczytaniu należy usunąć policzone wartości ze wszystkich komórek typu "code" (z użyciem `map`), a także kod z kratek typu "code" znajdujących się bezpośrednio po kratkach typu "markdown" zawierających w pierwszej linii ciąg `# Ćwiczenie`. Do tego drugiego usuwania użyj `reduce` z odpowiednią strukturą "zbierającą". Po usunięciu należy zapisać całość do nowego pliku `.ipynb`. Jego nazwę utwórz poprzez przekształcenie nazwy piku wejściowego, np. `plik.ipynb` -> `plik.czysty.ipynb`. Plik wynikowy powinien dać się otworzyć w Jupyter Notebook albo Jupyter Lab :)

Dla przykładowego pliku pliku wejściowego `plik.ipynb` (zawartość poglądowa):
```
<kratka markdown>
   Ten program wypisuje "A kuku".
<kratka code>
   <kod> print("A kuku")
   <wyjście> A kuku
<kratka markdown>
   ### Ćwiczenie 1
   Napisz program wypisujący "Ala ma kota".
<kratka code>
   <kod> print("Ala ma kota")
   <wyjście> Ala ma kota
```
należy utworzyć plik wyjściowy `plik.czysty.ipynb` (w porządnym formacie!) zawierający
```
<kratka markdown>
   Ten program wypisuje "A kuku".
<kratka code>
   <kod> print("A kuku")
   <wyjście>
<kratka markdown>
   ### Ćwiczenie 1
   Napisz program wypisujący "Ala ma kota".
<kratka code>
   <kod>
   <wyjście>
```
Szczegóły specyfikacji zadania należy sobie dopracować samemu poprzez inspekcję pliku notebooka z dzisiejszych zajęć (osobiście uzupełnionego o elementy przeznaczone do usunięcia).